In [1]:
import numpy as np

from new_get_reflected_1d import flux_get_reflected_1d, get_reflected_1d_original
from gfluxv_pyversion import get_reflected_1d_gfluxv

In [2]:
### Benchmarking things between these three routines

## flux_get_reflected_1d gets you level upwelling and downwelling scattering post-processed fluxes 

## get_reflected_1d_original is the original PICASO routine just intensity multiplied with pi to get upwelling fluxes 

## get_reflected_1d_gfluxv is the py version of fortran routine 
## -- it gives you upwelling and downwelling fluxes no post processed 

## now TEST input numbers

numg,numt=1,1 # of angles

nwno=3 # wvpoints

wno=np.array([1.,2.,3.]) # wavelengths

nlevel=11    # of levels
nlayer=nlevel-1 # of layers

tau=np.zeros((nlevel,nwno)) # cumulative tau array
dtau=np.zeros((nlevel-1,nwno)) # per layer tau array
w0,cosb=np.zeros((nlevel-1,nwno)),np.zeros((nlevel-1,nwno)) # ssa and asymmetry
w0+=0.1 # lets assume
cosb+=1e-3 # lets assume
for i in range(nlevel):
    for j in range(nwno):
        tau[i,j]=i+j # just an assumption : tau must always increase downwards

taucld=tau*0+0.1 ## needed for the scattering routines -- not get_reflected_1d_gfluxv
tauray=tau*0+0.1 ## needed for the scattering routines -- not get_reflected_1d_gfluxv



for i in range(nlevel-1):
    dtau[i,:]=tau[i+1,:]-tau[i,:] # incremental opd
 

surf_reflect=1.0 

ubar0=np.zeros((numg,numt))

ubar0+=1.0 # cosine(0)
ubar1=ubar0 #assumed

F0PI = np.zeros(nwno) + 1.0  # assumed


b_top = 0.0   #needed for  get_reflected_1d_gfluxv  as I take it as an input as in GFLUXV                                  
#needed for  get_reflected_1d_gfluxv  as I take it as an input as in GFLUXV
b_surface = 0. + surf_reflect*ubar0[0, 0]*F0PI*np.exp(-tau[-1, :]/ubar0[0, 0]) 


tridiagonal=0

## delta eddington correction for flux_get_reflected_1d, get_reflected_1d_original

tau_og=np.zeros((nlevel,nwno))
dtau_og=np.zeros((nlevel-1,nwno))
w0_og,cosb_og=np.zeros((nlevel-1,nwno)),np.zeros((nlevel-1,nwno))

dtau_og=dtau*(1.-w0*cosb**2)
tau_og[0]=tau[0]*(1.-w0[0]*cosb[0]**2)
for i in range(nlayer):
    tau_og[i+1]=tau[i]+dtau[i]
w0_og=w0*((1.-cosb**2)/(1.-w0*(cosb**2)))
cosb_og=cosb/(1.+cosb)

### now calculate ftau_cld and ftau_ray for flux_get_reflected_1d, get_reflected_1d_original (post-processing routines)
taucld_layer=0.5*(taucld[:-1,:]+taucld[1:,:])
tauray_layer=0.5*(tauray[:-1,:]+tauray[1:,:])

ftau_cld=taucld_layer/(taucld_layer+tauray_layer)
ftau_ray=tauray_layer/(taucld_layer+tauray_layer)

gcos2=0.5*ftau_ray
cos_theta=1.0

## phase function assumptions
frac_a,frac_b,frac_c=1,-1,2
constant_back, constant_forward=0.5,0.5
single_phase=3 ##TTHG_ray
multi_phase=0  ## N=2

In [3]:
flux_minus_all, flux_plus_all, flux_minus_midpt_all,flux_plus_midpt_all=flux_get_reflected_1d(nlevel, wno,nwno, numg,numt, dtau, tau, w0, cosb,gcos2, ftau_cld, ftau_ray,
    dtau_og, tau_og, w0_og, cosb_og, surf_reflect,ubar0, ubar1,cos_theta,F0PI,single_phase, multi_phase,
    frac_a, frac_b, frac_c, constant_back, constant_forward, tridiagonal, calc_type="climate")

In [4]:
flux_plus_all_original=get_reflected_1d_original(nlevel, wno,nwno, numg,numt, dtau, tau, w0, cosb,gcos2, ftau_cld, ftau_ray,
    dtau_og, tau_og, w0_og, cosb_og, surf_reflect,ubar0, ubar1,cos_theta,F0PI,single_phase, multi_phase,
    frac_a, frac_b, frac_c, constant_back, constant_forward, tridiagonal)

In [5]:
### difference in downwelling fluxes between the two post-processed routines (original PICASO and new post-processed)



print(flux_plus_all-flux_plus_all_original)

## so our new routine does well with upwelling flux component

[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]


In [6]:
flux_minus_all_gfluxv, flux_plus_all_gfluxv, flux_minus_midpt_all_gfluxv, flux_plus_midpt_all_gfluxv=get_reflected_1d_gfluxv(nlevel, wno,nwno, numg,numt, dtau, tau, w0, cosb,
    surf_reflect,b_top,b_surface,ubar0, F0PI,tridiagonal=0,delta_approx=1)

In [7]:
### difference in downwelling level fluxes between post-processed and not post-processed routines

print((flux_minus_all-flux_minus_all_gfluxv)/flux_minus_all_gfluxv)


[[[[ 0.00000000e+00 -9.99999949e-08 -1.99999980e-07]
   [-2.22496758e-02 -2.22497736e-02 -2.22498713e-02]
   [-2.61679978e-02 -2.61680952e-02 -2.61681926e-02]
   [-2.11053140e-02 -2.11054118e-02 -2.11055097e-02]
   [-1.14569421e-02 -1.14570410e-02 -1.14571398e-02]
   [ 5.83086580e-04  5.82986521e-04  5.82886463e-04]
   [ 1.38853679e-02  1.38852665e-02  1.38851651e-02]
   [ 2.78743492e-02  2.78742464e-02  2.78741437e-02]
   [ 4.24718546e-02  4.24717504e-02  4.24716462e-02]
   [ 6.07840706e-02  6.07839647e-02  6.07838588e-02]
   [-7.69244555e-07 -8.66807996e-07 -9.64371427e-07]]]]


In [8]:
### difference in upwelling level fluxes between post-processed and not post-processed routines

print((flux_plus_all-flux_plus_all_gfluxv)/flux_plus_all_gfluxv)


[[[[-1.38574636e-01 -1.38574652e-01 -1.38574669e-01]
   [-1.41232545e-01 -1.41232631e-01 -1.41232717e-01]
   [-1.42522297e-01 -1.42522383e-01 -1.42522469e-01]
   [-1.43137582e-01 -1.43137668e-01 -1.43137753e-01]
   [-1.43186006e-01 -1.43186092e-01 -1.43186177e-01]
   [-1.41270635e-01 -1.41270721e-01 -1.41270807e-01]
   [-1.26497099e-01 -1.26497186e-01 -1.26497273e-01]
   [-2.63312804e-02 -2.63313761e-02 -2.63314718e-02]
   [ 4.68321122e-01  4.68321004e-01  4.68320885e-01]
   [ 7.40918672e-01  7.40918626e-01  7.40918581e-01]
   [ 1.32490315e-07  1.25100306e-07  1.17710297e-07]]]]


In [9]:
### difference in downwelling layer fluxes between post-processed and not post-processed routines

print((flux_minus_midpt_all-flux_minus_midpt_all_gfluxv)/flux_minus_midpt_all_gfluxv)

[[[[ 7.17758066e-09 -9.28224151e-08 -1.92822400e-07]
   [-1.10273137e-02 -1.10274126e-02 -1.10275115e-02]
   [-8.80339535e-03 -8.80349447e-03 -8.80359359e-03]
   [ 7.64665456e-05  7.63665380e-05  7.62665304e-05]
   [ 1.23998277e-02  1.23997265e-02  1.23996252e-02]
   [ 2.65320416e-02  2.65319390e-02  2.65318363e-02]
   [ 4.16226512e-02  4.16225470e-02  4.16224429e-02]
   [ 5.72333739e-02  5.72332682e-02  5.72331625e-02]
   [ 7.32650386e-02  7.32649313e-02  7.32648241e-02]
   [ 9.14351889e-02  9.14350805e-02  9.14349721e-02]]]]


In [10]:
### difference in upwelling layer fluxes between post-processed and not post-processed routines

print((flux_plus_midpt_all-flux_plus_midpt_all_gfluxv)/flux_plus_midpt_all_gfluxv)

[[[[-1.28559369e-01 -1.28559456e-01 -1.28559543e-01]
   [-9.46056768e-02 -9.46057673e-02 -9.46058579e-02]
   [-7.76888105e-02 -7.76889027e-02 -7.76889949e-02]
   [-6.89716817e-02 -6.89717748e-02 -6.89718679e-02]
   [-6.39609225e-02 -6.39610161e-02 -6.39611097e-02]
   [-5.78492424e-02 -5.78493366e-02 -5.78494308e-02]
   [-3.09929601e-02 -3.09930566e-02 -3.09931531e-02]
   [ 1.18464621e-01  1.18464516e-01  1.18464411e-01]
   [ 4.32511315e-01  4.32511238e-01  4.32511161e-01]
   [ 6.12976386e-08  4.92222412e-08  3.71468432e-08]]]]
